In [1]:
!git clone https://github.com/ospanbatyr/comp_seq2seq_lstm.git

Cloning into 'comp_seq2seq_lstm'...
remote: Enumerating objects: 547, done.
remote: Counting objects: 100% (547/547), done.
remote: Compressing objects: 100% (279/279), done.
remote: Total 547 (delta 263), reused 505 (delta 221), pack-reused 0
Receiving objects: 100% (547/547), 13.10 MiB | 13.79 MiB/s, done.
Resolving deltas: 100% (263/263), done.


In [2]:
!pip install -r /content/drive/MyDrive/DataLab/turkeyproject/Code/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.9 MB/s 
     |████████████████████████████████| 7.6 MB 14.9 MB/s 
     |████████████████████████████████| 5.8 MB 31.2 MB/s 
     |████████████████████████████████| 125 kB 44.7 MB/s 
     |████████████████████████████████| 588.3 MB 20 kB/s 
     |████████████████████████████████| 182 kB 40.9 MB/s 
     |████████████████████████████████| 439 kB 54.1 MB/s 
     |████████████████████████████████| 6.0 MB 36.4 MB/s 
     |████████████████████████████████| 1.7 MB 28.9 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1

In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# !wandb sync /content/wandb/offline-run-20221121_191113-3r4yuq1s
# !wandb login --relogin
# AIP: ebb0b79222e608e8bc1eb3a650f1e1da50fda72e

!wandb login

wandb: Currently logged in as: taninzrt. Use `wandb login --relogin` to force relogin


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !cp /content/drive/MyDrive/DataLab/turkeyproject/Code/src/model_t.py /content
# 
!cp /content/drive/MyDrive/DataLab/turkeyproject/Code/src/main_cls.py /content


drive.MyDrive.DataLab.turkeyproject.Code.src

In [ ]:
!python /content/main_cls.py -mode train -project_name test_runs_cls -hidden_size 64 -model_selector_set val -pretrained_model_name none -finetune_data_voc none -dev_set -no-test_set -no-gen_set -dataset simple_split_tsv_cls -dev_always -no-test_always -no-gen_always -epochs 10 -save_model -show_train_acc -embedding random -no-freeze_emb -no-freeze_emb2 -no-freeze_lstm_encoder -no-freeze_lstm_decoder -no-freeze_fc -batch_size 256 -lr 0.008 -emb_lr 0.001 -dropout 0.1 -no_beam_decode -run_name RUN-train_try_cls -gpu 1 -topk 1 

2022-12-06 15:29:16.942972: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-12-06 15:29:16.943115: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-12-06 15:29:16.943141: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
wandb: Currently logged in as: taninzrt. Use `wandb login --relogin` to force relogin
wandb: Tracking run with 


# Load Logits

## Import

In [18]:
import os
import numpy as np
import pandas as pd
import torch
import tensorflow as tf

In [19]:
df = pd.read_excel('/content/drive/MyDrive/DataLab/turkeyproject/Code/outputs/output.xlsx')

In [20]:
df.head()

,Unnamed: 0,src,target,epoch,label,idx
0,0,look opposite left thrice and jump opposite ri...,I_TURN_LEFT I_TURN_LEFT I_LOOK I_TURN_LEFT I_T...,1,1,11385
1,1,run twice after turn opposite left,I_TURN_LEFT I_TURN_LEFT I_RUN I_WALK,1,0,1789
2,2,run opposite left and turn around right twice,I_TURN_LEFT,1,0,7253
3,3,walk around left thrice and turn opposite left...,I_TURN_LEFT I_WALK I_TURN_LEFT I_WALK I_TURN_L...,1,0,11553
4,4,walk right thrice and walk around left twice,I_TURN_RIGHT I_WALK I_TURN_RIGHT I_WALK I_TURN...,1,1,8066


In [22]:
def load_logits(dir_path: str):
    file_list = os.listdir(dir_path)
    file_list.sort()
    print("Loading files in:", dir_path)
    labels, idxs, logits = [], [], [], 
    for file_name in file_list:
        file_path = f"{dir_path}/{file_name}"
        if "idxs" in file_path:
            idxs.append(np.array(torch.load(file_path)))
        elif "logits" in file_path:
            logits.append(np.array(torch.load(file_path)))
        elif "labels" in file_path:
            labels.append(np.array(torch.load(file_path)))
         
                     
        else:
            raise Exception("Wrong Files!")
    logits_ordered = np.zeros(np.array(logits).shape)
    
    
    for epoch in range(len(idxs)):
      # print(idxs[epoch])
      logits_ordered[epoch][idxs[epoch]] = logits[epoch]

      # true_labels[epoch][idxs[epoch]] = labels[epoch]
    
    return logits_ordered, labels, idxs


In [23]:
logits, labels, idx = load_logits('/content/drive/MyDrive/DataLab/turkeyproject/Code/outputs/cartography')

Loading files in: /content/drive/MyDrive/DataLab/turkeyproject/Code/outputs/cartography


In [24]:
# true_labels =torch.from_numpy(labels)
true_labels = torch.Tensor(labels)

<ipython-input-24-adb8ef613dd8>:2: UserWarning:

Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)



In [25]:
true_labels

tensor([[0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 1.]])

In [26]:
# true_labels2 = torch.tensor(true_labels, dtype=torch.int64)

true_labels2 = true_labels.type(torch.int64)

In [27]:
logits_t = torch.Tensor(logits)

type(logits_t)

torch.Tensor

In [28]:
print(true_labels2)

tensor([[0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 1, 1, 1],
        ...,
        [1, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 0, 1]])


In [31]:
def cartography(logits, true_labels):
    
    probs = torch.nn.functional.softmax(logits, dim=-1)
    predictions = probs.numpy()
    
    corr_probs = np.sum(predictions * np.expand_dims(torch.nn.functional.one_hot(true_labels, num_classes=2).numpy(), axis=0), axis=-1)
    
    confs = np.mean(corr_probs[0], axis=0)
    
    variabilities = np.std(corr_probs[0], axis=0)
    
    return confs, variabilities

In [32]:
conf, vari = cartography(logits_t, true_labels2)

[[[0.50750965 0.50750965 0.50750965 ... 0.44773248 0.10619404 0.72055233]
  [0.54325491 0.50750965 0.50750965 ... 0.76023668 0.80131817 0.88223279]
  [0.50750965 0.50750965 0.49249029 ... 0.45050764 0.34691879 0.15091173]
  ...
  [0.49249029 0.50750965 0.50750965 ... 0.47226149 0.82174885 0.15490626]
  [0.50750965 0.50750965 0.50750965 ... 0.40152028 0.84410268 0.69974518]
  [0.50750965 0.50750965 0.50750965 ... 0.55737078 0.70069546 0.09588663]]]


In [69]:
def correctness(logits, true_labels):
  correctness = []
  epochs = len(true_labels)

  probs = torch.nn.functional.softmax(logits, dim=-1)
  predictions = probs.numpy()
  
  corr_probs = np.sum(predictions * np.expand_dims(torch.nn.functional.one_hot(true_labels, num_classes=2).numpy(), axis=0), axis=-1)

  y = torch.argmax(probs, dim=-1)
  # print(y)
  for i in range(len(y[0])):
    correct = 0
    for j in range(len(y)):
      if y[j][i] == true_labels[j][i]:
        correct += 1
    correctness.append(correct/epochs)

  return correctness

In [70]:
correctness = correctness(logits_t, true_labels2)

In [71]:
correctness

[0.9,
 0.9,
 0.9,
 0.7,
 0.8,
 1.0,
 0.7,
 0.5,
 0.9,
 0.7,
 0.9,
 0.8,
 0.9,
 0.9,
 0.6,
 0.6,
 0.8,
 0.7,
 0.8,
 0.9,
 0.6,
 0.6,
 0.8,
 0.6,
 0.8,
 0.7,
 0.9,
 0.8,
 1.0,
 0.5,
 0.8,
 0.6,
 0.6,
 0.7,
 0.9,
 0.6,
 0.8,
 0.8,
 0.7,
 0.9,
 0.7,
 0.6,
 1.0,
 1.0,
 0.9,
 0.5,
 0.8,
 0.5,
 1.0,
 0.6,
 0.6,
 0.6,
 0.7,
 0.6,
 0.7,
 0.8,
 1.0,
 0.6,
 0.7,
 0.7,
 0.8,
 0.6,
 0.8,
 0.8,
 0.7,
 0.5,
 1.0,
 0.8,
 0.5,
 0.5,
 0.7,
 0.9,
 0.7,
 0.8,
 0.6,
 0.9,
 0.9,
 0.8,
 0.5,
 0.9,
 0.9,
 0.8,
 0.8,
 0.5,
 0.8,
 0.8,
 0.7,
 0.7,
 0.7,
 0.6,
 0.6,
 0.7,
 0.7,
 0.8,
 0.8,
 0.7,
 0.8,
 0.8,
 0.7,
 0.5,
 0.6,
 0.5,
 0.8,
 0.9,
 0.5,
 0.7,
 0.6,
 0.8,
 0.7,
 0.7,
 0.9,
 0.8,
 0.7,
 0.7,
 0.8,
 0.8,
 0.8,
 1.0,
 0.7,
 0.8,
 1.0,
 0.0,
 0.9,
 0.7,
 0.8,
 0.8,
 0.6,
 0.8,
 0.8,
 0.7,
 0.9,
 0.7,
 0.8,
 0.6,
 0.8,
 0.8,
 1.0,
 0.5,
 0.9,
 0.8,
 0.5,
 1.0,
 0.7,
 1.0,
 0.8,
 0.9,
 0.9,
 0.8,
 0.8,
 0.8,
 0.7,
 0.8,
 0.7,
 0.7,
 0.7,
 1.0,
 1.0,
 0.9,
 0.7,
 0.8,
 0.8,
 0.6,
 0.9,
 0.7,
 0.5,
 0.6,
 0.5

## Plot

In [ ]:
import matplotlib.pyplot as plt 

# x-axis values
x = vari
# y-axis values
y = conf

# plotting points as a scatter plot
plt.scatter(x, y, label= "stars", color= "green",
      marker= ".", s=30)

# x-axis label
plt.xlabel('x - variability')
# frequency label
plt.ylabel('y - confidence')
# plot title
plt.title('Dataset Cartography')
# showing legend
plt.legend()
# plt.xlim([-0.00000001, 0.00000001])
# function to show the plot
plt.show()


# Save Data

In [72]:
new_df = pd.DataFrame({
    'confidence': conf,
    'variebility': vari,
    'correctness': correctness,
    'idx':range(0, len(conf))
}, columns= ['confidence', 'variebility', 'correctness','idx'])

In [73]:
new_df.tail()

,confidence,variebility,correctness,idx
12708,0.597249,0.078635,1.0,12708
12709,0.634566,0.317472,0.7,12709
12710,0.505505,0.094436,0.4,12710
12711,0.567301,0.306127,0.6,12711
12712,0.540360,0.333806,0.6,12712


In [74]:
new_df.head()

,confidence,variebility,correctness,idx
0,0.509582,0.012084,0.9,0
1,0.537128,0.093969,0.9,1
2,0.514428,0.020217,0.9,2
3,0.529448,0.081113,0.7,3
4,0.542257,0.063037,0.8,4


In [75]:
cartography = pd.merge(df, new_df, on="idx")

In [76]:
cartography.head()

,Unnamed: 0,src,target,epoch,label,idx,confidence,variebility,correctness
0,0,look opposite left thrice and jump opposite ri...,I_TURN_LEFT I_TURN_LEFT I_LOOK I_TURN_LEFT I_T...,1,1,11385,0.636419,0.155860,0.8
1,1,run twice after turn opposite left,I_TURN_LEFT I_TURN_LEFT I_RUN I_WALK,1,0,1789,0.467465,0.204517,0.7
2,2,run opposite left and turn around right twice,I_TURN_LEFT,1,0,7253,0.589288,0.333396,0.7
3,3,walk around left thrice and turn opposite left...,I_TURN_LEFT I_WALK I_TURN_LEFT I_WALK I_TURN_L...,1,0,11553,0.530800,0.206831,0.6
4,4,walk right thrice and walk around left twice,I_TURN_RIGHT I_WALK I_TURN_RIGHT I_WALK I_TURN...,1,1,8066,0.504332,0.048566,0.7


In [77]:
cartography.to_excel('/content/drive/MyDrive/DataLab/turkeyproject/Code/outputs/cartographyOutput.xlsx')

Plot DataFrame

In [14]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px

In [79]:
df = pd.read_excel('/content/drive/MyDrive/DataLab/turkeyproject/Code/outputs/cartographyOutput.xlsx')

In [12]:
df.head()

,Unnamed: 0,Unnamed: 0.1,src,target,epoch,label,idx,confidence,variebility
0,0,0,look opposite left thrice and jump opposite ri...,I_TURN_LEFT I_TURN_LEFT I_LOOK I_TURN_LEFT I_T...,1,1,11385,0.636419,0.155860
1,1,1,run twice after turn opposite left,I_TURN_LEFT I_TURN_LEFT I_RUN I_WALK,1,0,1789,0.467465,0.204517
2,2,2,run opposite left and turn around right twice,I_TURN_LEFT,1,0,7253,0.589288,0.333396
3,3,3,walk around left thrice and turn opposite left...,I_TURN_LEFT I_WALK I_TURN_LEFT I_WALK I_TURN_L...,1,0,11553,0.530800,0.206831
4,4,4,walk right thrice and walk around left twice,I_TURN_RIGHT I_WALK I_TURN_RIGHT I_WALK I_TURN...,1,1,8066,0.504332,0.048566


In [16]:
px.scatter(df,x="variebility", y="confidence", color='label', color_discrete_sequence =['#ecef4a','#ff3c83'])

In [80]:
px.scatter(df,x="variebility", y="confidence", color='correctness')